<a href="https://colab.research.google.com/github/ygsriram/NLP/blob/master/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the required libraries

In [0]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

Installing the bert tensorflow library that conatins bert models

In [0]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.9MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


You can download the model folder from [here]("https://drive.google.com/open?export=download&id=1sWxHR3QPDg1uggGPrtkSOyZAIGPZivvx")

In [0]:
#change the path to your model folder
#/content/drive/My Drive/model.tar
#/content/drive/My Drive/Colab Notebooks/model.tar
!tar -xvf "/content/drive/My Drive/Colab Notebooks/model.tar"  -C ./

model3/
model3/model.ckpt-0.meta
model3/results
model3/model.ckpt-93.meta
model3/checkpoint
model3/.ipynb_checkpoints/
model3/model.ckpt-0.data-00000-of-00001
model3/model.ckpt-0.index
model3/model.ckpt-93.data-00000-of-00001
model3/model.ckpt-93.index
model3/graph.pbtxt


You can download the COPA dataset from [here](http://people.ict.usc.edu/~gordon/downloads/COPA-resources.tgz)

In [0]:
!tar -xvzf COPA-resources.tgz  -C ./

COPA-resources/
COPA-resources/copa-eval.sh
COPA-resources/results/
COPA-resources/results/PMIgutenbergW5.test
COPA-resources/results/baselineFirst.all
COPA-resources/results/baselineFirst.test
COPA-resources/results/PMIgutenbergW5.dev
COPA-resources/results/PMIgutenbergW5.all
COPA-resources/results/baselineFirst.dev
COPA-resources/results/gold.dev
COPA-resources/results/gold.all
COPA-resources/results/gold.test
COPA-resources/readme.txt
COPA-resources/copa-eval.jar
COPA-resources/datasets/
COPA-resources/datasets/copa-all.xml
COPA-resources/datasets/copa-dev.xml
COPA-resources/datasets/copa.dtd
COPA-resources/datasets/copa-test.xml


Parsing the dataset to create train and test data frames

In [0]:
def getdata(path_data,path_label):
  data = open(path_data).read()
  soup = BeautifulSoup(data, "html.parser")
  items = soup.findChildren(name="item")
  label = open(path_label).read()
  labels = label.split("\n")
  lab_a1 = []
  lab_a2 = []
  for l in labels:
    if l is not "":
      lab_a1.append(int(l.split(" ")[1]))
      lab_a2.append(int(l.split(" ")[2]))
  df = pd.DataFrame(columns=["cause","a","label"])
  for item,l1,l2 in zip(items,lab_a1, lab_a2):
    p = item.findChild(name="p").get_text()
    a1 = item.findChild(name="a1").get_text()
    a2 = item.findChild(name="a2").get_text()
    df = df.append({"cause":p,"a":a1,"label":l1},ignore_index=True)
    df = df.append({"cause":p,"a":a2,"label":l2},ignore_index=True)
  
  return df
  


In [0]:
train_data_path = "COPA-resources/datasets/copa-dev.xml"
train_label_path = "/content/COPA-resources/results/gold.dev"
df_train = getdata(train_data_path, train_label_path)

test_data_path = "COPA-resources/datasets/copa-test.xml"
test_label_path = "/content/COPA-resources/results/gold.test"
df_test = getdata(test_data_path, test_label_path)

In [0]:
display(df_train)
display(df_test)

,cause,a,label
0,My body cast a shadow over the grass.,The sun was rising.,1
1,My body cast a shadow over the grass.,The grass was cut.,0
2,The woman tolerated her friend's difficult beh...,The woman knew her friend was going through a ...,1
3,The woman tolerated her friend's difficult beh...,The woman felt that her friend took advantage ...,0
4,The women met for coffee.,The cafe reopened in a new location.,0
...,...,...,...
995,The airline mishandled my luggage.,They cancelled my flight.,0
996,The computer was expensive to fix.,I got it repaired.,0
997,The computer was expensive to fix.,I bought a new one.,1
998,The woman was in a bad mood.,She engaged in small talk with her friend.,0


,cause,a,label
0,The item was packaged in bubble wrap.,It was fragile.,1
1,The item was packaged in bubble wrap.,It was small.,0
2,I emptied my pockets.,I retrieved a ticket stub.,1
3,I emptied my pockets.,I found a weapon.,0
4,Termites invaded the house.,The termites disappeared from the house.,0
...,...,...,...
995,The traveler walked on the shaky suspension br...,He felt ecstatic.,0
996,The man anticipated the team's victory.,He met his friends to watch the game.,0
997,The man anticipated the team's victory.,He made a bet with his friends.,1
998,The boy couldn't fall asleep.,He set his alarm clock.,0


In [0]:
train = df_train
test = df_test

Creating the config json to hold all the hyperparameters

In [0]:
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
config = {"SENTENCE1" : 'cause',
          "SENTENCE2" :  "a",
          "LABEL_COLUMN" : 'label',
          "label_list" : [0, 1],
          "MAX_SEQ_LENGTH" : 64, #128
          "BATCH_SIZE" : 64, #32,
          "LEARNING_RATE" : 2e-5, #2e-5,
          "NUM_TRAIN_EPOCHS" : 3.0,
          "WARMUP_PROPORTION" : 0.1,
          "SAVE_CHECKPOINTS_STEPS" : 250, #500
          "SAVE_SUMMARY_STEPS" : 100,
          "OUTPUT_DIR" : "./model3/"}

Applying BERT embeddings on our data

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[config["SENTENCE1"]], 
                                                                   text_b = x[config["SENTENCE2"]], 
                                                                   label = x[config["LABEL_COLUMN"]]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[config["SENTENCE1"]], 
                                                                   text_b = x[config["SENTENCE2"]], 
                                                                   label = x[config["LABEL_COLUMN"]]), axis = 1)

Applying BERT tokenizer on our data

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)
tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Converting our tokens into features

In [0]:
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, config["label_list"], config["MAX_SEQ_LENGTH"], tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, config["label_list"], config["MAX_SEQ_LENGTH"], tokenizer)

INFO:tensorflow:Writing example 0 of 1000


INFO:tensorflow:Writing example 0 of 1000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


Creating a model to train and evaluate

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.75)
    output_layer = tf.layers.Dense(hidden_size)(output_layer)


    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)
    
    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    """#https://stackoverflow.com/questions/36904298/how-to-implement-multi-class-hinge-loss-in-tensorflow
    H = tf.reduce_max(one_hot_labels * log_probs, -1)    
    L = tf.nn.relu((1 - log_probs + H) * one_hot_labels)
    loss = tf.reduce_mean(tf.reduce_max(L, 0))"""  

    return (loss, predicted_labels, log_probs)


Creating a model builder to train using tf estimators

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int((len(train_features) / config["BATCH_SIZE"]) * config["NUM_TRAIN_EPOCHS"])
num_warmup_steps = int(num_train_steps * config["WARMUP_PROPORTION"])

In [0]:
# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=config["OUTPUT_DIR"],
    save_summary_steps=config["SAVE_SUMMARY_STEPS"],
    save_checkpoints_steps=config["SAVE_CHECKPOINTS_STEPS"])

Building the model function and estimator

In [0]:
model_fn = model_fn_builder(
  num_labels=len(config["label_list"]),
  learning_rate=config["LEARNING_RATE"],
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": config["BATCH_SIZE"]})


INFO:tensorflow:Using config: {'_model_dir': './model3/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 250, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fce9f34df28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './model3/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 250, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fce9f34df28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=config["MAX_SEQ_LENGTH"],
    is_training=True,
    drop_remainder=False)

Training the model

In [0]:
# uncomment this cell only to train the model
"""print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps, saving_listeners = tf.keras.callbacks.TensorBoard("./model3", histogram_freq=1)
 )
print("Training took time ", datetime.now() - current_time)"""

'print(f\'Beginning Training!\')\ncurrent_time = datetime.now()\nestimator.train(input_fn=train_input_fn, max_steps=num_train_steps, saving_listeners = tf.keras.callbacks.TensorBoard("./model3", histogram_freq=1)\n )\nprint("Training took time ", datetime.now() - current_time)'

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=config["MAX_SEQ_LENGTH"],
    is_training=False,
    drop_remainder=False)

Testing the model

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 93: auc = 0.663, eval_accuracy = 0.663, f1_score = 0.6841611, false_negatives = 135.0, false_positives = 202.0, global_step = 93, loss = 0.6597916, precision = 0.643739, recall = 0.73, true_negatives = 298.0, true_positives = 365.0


INFO:tensorflow:Saving dict for global step 93: auc = 0.663, eval_accuracy = 0.663, f1_score = 0.6841611, false_negatives = 135.0, false_positives = 202.0, global_step = 93, loss = 0.6597916, precision = 0.643739, recall = 0.73, true_negatives = 298.0, true_positives = 365.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 93: ./model3/model.ckpt-93


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 93: ./model3/model.ckpt-93


{'auc': 0.663,
 'eval_accuracy': 0.663,
 'f1_score': 0.6841611,
 'false_negatives': 135.0,
 'false_positives': 202.0,
 'global_step': 93,
 'loss': 0.6597916,
 'precision': 0.643739,
 'recall': 0.73,
 'true_negatives': 298.0,
 'true_positives': 365.0}

References
- https://arxiv.org/pdf/1810.04805.pdf
- http://people.ict.usc.edu/~gordon/copa.html
- https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb#scrollTo=6o2a5ZIvRcJq
- https://www.crummy.com/software/BeautifulSoup/

In [0]:
def getPrediction(premise, hypothesis):
  labels = [0, 1]
  input_examples = [run_classifier.InputExample(guid="", text_a = x1, text_b = x2, label = 0) for x1,x2 in zip(premise, hypothesis)] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, config["label_list"], config["MAX_SEQ_LENGTH"], tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=config["MAX_SEQ_LENGTH"], is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence1, sentence2, prediction['probabilities'], labels[prediction['labels']]) for sentence1, sentence2, prediction in zip(premise, hypothesis, predictions)]

In [0]:
premise = df_test.loc[100:120].cause
hypothesis = df_test.loc[100:120].a
label = df_test.loc[100:120].label

In [0]:
predictions = getPrediction(premise, hypothesis)

INFO:tensorflow:Writing example 0 of 21


INFO:tensorflow:Writing example 0 of 21


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from ./model3/model.ckpt-93


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
predictions

[('The little boy cried to his babysitter.',
  'He missed his parents.',
  array([-2.1456401 , -0.12442225], dtype=float32),
  1),
 ('The little boy cried to his babysitter.',
  'It was snack time.',
  array([-0.09592264, -2.3917916 ], dtype=float32),
  0),
 ("The pilot's radar detected a storm.",
  'The pilot navigated away from the storm.',
  array([-1.4275334, -0.2743052], dtype=float32),
  1),
 ("The pilot's radar detected a storm.",
  'The pilot flew through the storm.',
  array([-2.075646  , -0.13407484], dtype=float32),
  1),
 ('The tree shed its leaves.',
  'The leaves turned colors.',
  array([-0.5748444, -0.8273473], dtype=float32),
  0),
 ('The tree shed its leaves.',
  'The leaves accumulated on the ground.',
  array([-0.5073656, -0.9215035], dtype=float32),
  0),
 ('The boy was in a mischievous mood.',
  'He decided to play a card game with his sister.',
  array([-1.1236198 , -0.39319196], dtype=float32),
  1),
 ('The boy was in a mischievous mood.',
  'He decided to play 

In [0]:
df_test.loc[100:120]

,cause,a,label
100,The little boy cried to his babysitter.,He missed his parents.,1
101,The little boy cried to his babysitter.,It was snack time.,0
102,The pilot's radar detected a storm.,The pilot navigated away from the storm.,1
103,The pilot's radar detected a storm.,The pilot flew through the storm.,0
104,The tree shed its leaves.,The leaves turned colors.,0
105,The tree shed its leaves.,The leaves accumulated on the ground.,1
106,The boy was in a mischievous mood.,He decided to play a card game with his sister.,0
107,The boy was in a mischievous mood.,He decided to play a practical joke on his sis...,1
108,The child complained that he had to go to the ...,His father gave him a soda to drink.,0
109,The child complained that he had to go to the ...,His father stopped the car at a gas station.,1
